In [1]:
import os
import numpy as np
import math, argparse
import scipy.optimize
from scipy.integrate import cumtrapz
from scipy.interpolate import CubicSpline
from collections import defaultdict
import matplotlib.pyplot as plt

In [40]:
def mayer_function(r_array,v_array):
    y = np.zeros(np.shape(v_array))
    for i in range(np.shape(r_array)[0]):
        y[i] = 2.0 * math.pi * r_array[i]**2 * (1. - math.exp(-v_array[i]/kb/T))
    return y

def B2_function(mayer_data,r):
    return cumtrapz(mayer_data, r)


def fit_pmf(dr, pmf, pmf_err, gyrsuminf, alpha=0.1, npoints_fine=1000):
    # Solve a Bayesian model for the Mayer f function.  Prior is a Gaussian envelope for the pmf.
    # Note that the distance scale in the gaussian argument should be scaled to account for real Rg.
    dr_mean = dr
    #print(dr[:-1])
    def prior_var(r):
        maxv = max(np.fabs(pmf))
        gaussian_envelope_r = maxv * math.exp(-1. * (r / gyrsuminf)**2)
        return (r**2 * (math.exp(gaussian_envelope_r) - 1.))**2
    def data_var(i):
        return (dr_mean[i]**2 * math.exp(-pmf[i]) * pmf_err[i])**2
    def mayer_r2(r, v):
        return r**2 * (math.exp(-v) - 1.)
  
    def get_v(r, f):
        return -math.log(f / r**2 + 1.)
  
    def predict_fine(cs):
        return (np.array([r for r in np.linspace(0., dr[-1], npoints_fine)]), \
                np.array([cs(r) for r in np.linspace(0., dr[-1], npoints_fine)]))
    def B2(cs):
        return scipy.integrate.quad(lambda r: 2.*math.pi*r**2 * (1. - math.exp(-cs(r))), 0, np.inf)[0]

    mayer_data = np.array([mayer_r2(dr_mean[i], pmf[i]) for i in range(len(pmf))])
    var_data = np.array([data_var(i) for i in range(len(pmf))])
    var_model = np.array([prior_var(dr_mean[i]) for i in range(len(pmf))])
    mayer_model = mayer_data / (1. + alpha * (var_data / var_model))
    pmf_model = np.array([get_v(dr_mean[i], mayer_model[i]) for i in range(len(pmf))])
    for i in range(len(pmf)):
        if math.fabs(pmf_model[i]) < 1.e-3:
            pmf_model[i] = 0.
    pmf_model_cs = CubicSpline(dr_mean, pmf_model, bc_type=('not-a-knot', 'clamped'))


    def cs_extrap(r):
        if r < dr_mean[-1] :
            return pmf_model_cs(r)
        else:
            return 0
    return None, pmf_model, predict_fine(pmf_model_cs), B2(cs_extrap)


In [44]:
kb = 0.001985875 #Boltzman constant kcal/(mol K)
T = 300 # Temperature K
bin_width = 1.0
nzero = 20
gyrinfsum = 20 # this is the radius of gyration, which should be determined by the one-single chain simulation 

pmf_name = 'pmf_ave_300.txt' # potential of mean force

pmf_ave = np.loadtxt(pmf_name)
dr = pmf_ave[:,0]
pmf_err = np.loadtxt('pmf_ave_err300.txt') # the standard deviation of potential of mean force

pmf_fit_params, pmf_fit, pmf_fit_fine, B2 = fit_pmf(dr, pmf_ave[:,1], pmf_err[:,1], gyrinfsum) # optimize the pmf_ave profile by using bayesian optimization

print('B2 is ', B2)
#plt.plot(pmf_ave[:,0], pmf_ave[:,1])
#plt.plot(pmf_ave[:,0], pmf_fit)
#plt.plot(pmf_fit_fine[0], pmf_fit_fine[1])
#plt.show()

B2 is  -501630.297015704


<ipython-input-40-f8a102c22d21>:32: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  return scipy.integrate.quad(lambda r: 2.*math.pi*r**2 * (1. - math.exp(-cs(r))), 0, np.inf)[0]
